# Pose Estimation based on Segmentation and Iterative Closest Point fit 

Please install any dependencies for the notebook (see bellow). It is best to use Python 3.8 interpreter and a virtual environment (see `venv` module).

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import open3d as o3d
from ultralytics import SAM
from ultralytics import RTDETR
import copy
import timeit

## 1. Read images

First we read the RGB and depth maps from example files. The camera used to record these images is a Intel D435 RGBD camera, with a known intrinsic matrix (i.e. `Kdepth`).

In [ ]:
color_img = cv2.imread('../asset/example_image.png')
depth_img = cv2.imread('../asset/example_depth.png', cv2.IMREAD_ANYDEPTH)
Kdepth = np.array([[637.22601318,   0.        , 644.54681396],
                    [  0.        , 636.76959229, 363.35079956],
                    [  0.        ,   0.        ,   1.        ]])

In [ ]:
print(f'Color image: {color_img.dtype, color_img.shape}')
print(f'Depth image: {depth_img.dtype, depth_img.shape}')

Visualize the images.

In [ ]:
plt.imshow(color_img)

In [ ]:
plt.imshow(depth_img)

Create the Point cloud using the $K$ matrix and the depth image.

In [ ]:
# create point cloud
pts = cv2.rgbd.depthTo3d(depth_img, Kdepth)

Visualize the point cloud using Open3D viewer. Please note, that the point cloud contains a lot of garbage points resulted from the violation of the minimum and maximum distance of the D435 camera.

In [ ]:
# visualize point cloud
verts = pts.reshape((-1,3))
idx = ~np.isnan(verts).any(axis=1)
verts = verts[idx,:]
color = color_img.reshape((-1,3))
color = color[idx,:]

pcd = o3d.geometry.PointCloud()
orig_pcd = pcd
pcd.points = o3d.utility.Vector3dVector(verts)
pcd.colors = o3d.utility.Vector3dVector(color/255)
o3d.visualization.draw_geometries([pcd])

## 2. Detection

First we find bottles in the RGB image using RTDETR object detector. After that, we save the detections and use the SAM model to segment the pixels of the detections, resulting in **masks** for the bottles we are interested in.

In [ ]:
det_model = RTDETR('rtdetr-x.pt')

In [ ]:
# get cls indices for bottle and cup
cls_idxs = [id for id,name in det_model.names.items() if name in ['bottle','cup']]
cls_idxs

In [ ]:
results = det_model(color_img, classes = cls_idxs)
det_result = results[0]
len(det_result)

In [ ]:
plt.imshow(det_result.plot())

In [ ]:
sam = SAM('mobile_sam.pt')

In [ ]:
sam_result = sam.predict(color_img, bboxes = det_result.boxes.xyxy)[0]

In [ ]:
plt.imshow(sam_result.plot())

## 3. Calculate the poses for the objects

Define `DOWN_SAMPLE_SIZE`. We will use this to downsample the point clouds to 4 mm in order to speed up ICP process.

In [ ]:
DOWN_SAMPLE_SIZE = 4e-3

Load the reference point cloud for the object. The pose will be calculated as a transformation between the reference object and the detected objects.

In [ ]:
ref_pcd = o3d.io.read_point_cloud("../asset/bottle_large.pcd")
cnt = np.asarray(ref_pcd.points).shape[0]
ref_pcd.colors = o3d.utility.Vector3dVector(np.repeat([[1,0,0]],cnt,axis = 0).astype(np.float32))
ref_pcd = ref_pcd.voxel_down_sample(DOWN_SAMPLE_SIZE)

A function to estimate the pose for an object in the RGBD image filtered by a mask.

In [ ]:
def estimate_pose_for_mask(pts,color_img,mask):
    """
    Estimate the pose for an object in the point cloud.
    - pts: the point cloud with shape (height,width,coord) with type np.float32
    - color_img: the colors for the pts points with shape (height,width,3)
    - mask: mask for the object with shape (height,width) with type np.bool

    Returns a tuple:
        tvec - translation
        rvec - Rodrigues vector for rotation
        reg_p2p - the result object from Open3D (open3d.pipelines.registration.RegistrationResult)
    """
    objpts = pts[mask,:]
    color = color_img[mask,:]

    #remove NaNs (e.g. wrong depth pixels)
    verts = objpts.reshape((-1,3))
    idx = ~np.isnan(verts).any(axis=1)
    verts = verts[idx,:]
    color = color[idx,:]

    # create PCD
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(verts)
    pcd.colors = o3d.utility.Vector3dVector(color/255)

    # downsample the PCD
    pcd_ds = pcd.voxel_down_sample(DOWN_SAMPLE_SIZE)

    # default transformation is around the mean of the object, with identity rotation
    pts_mean = np.mean(np.asarray(pcd_ds.points),axis=0)
    initial_transform = np.block([[np.identity(3), np.asmatrix(pts_mean).T],[0,0,0,1]])

    # run pose estimation with different outlier margins
    reg_p2p = o3d.pipelines.registration.registration_icp(
        ref_pcd, pcd_ds, 1, initial_transform,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))


    reg_p2p = o3d.pipelines.registration.registration_icp(
        ref_pcd, pcd_ds, 0.01, reg_p2p.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))
    
    # create rvec and tvec
    tvec = reg_p2p.transformation[0:3,3]
    rvec,_ = cv2.Rodrigues(reg_p2p.transformation[0:3,0:3])
    rvec = rvec.T[0,:]
    return tvec,rvec,reg_p2p


Calculate pose for all the objects detected in the image.

In [ ]:
results = [estimate_pose_for_mask(pts,color_img,mask.numpy()) for mask in sam_result.masks.data]

Visualize result, presenting the **fitness** score for the objects.

In [ ]:
# increase figure size for readable text
plt.rcParams['figure.figsize'] = [15, 7]

seg_debug_img = color_img.copy()

for id,(mask,result) in enumerate(zip(sam_result.masks,results)):
    mask = mask.data.numpy()[-1,:,:]

    blend = result[2].fitness
    seg_debug_img[mask,:] = (1 - blend) * seg_debug_img[mask,:] + (blend) * np.array([255,0,0])


for id,(mask,result) in enumerate(zip(sam_result.masks,results)):
    crd = np.mean(mask.xy,axis=1).astype(np.int32).flatten().tolist()
    text = f"{id} - {result[2].fitness:0.2} - {result[2].inlier_rmse:0.3}"
    cv2.putText(seg_debug_img,org=crd,text=text,fontFace = 0, fontScale = 0.4, color = (0,255,0), thickness = 1)

plt.imshow(seg_debug_img)

Filter the results by **fitness** score. Fitness ranges from 0 to 1, and shows the inlier proportion. For an object, even 0.5 can be a successful match, since the object can be seen from one side.

In [ ]:
FITNESS_LIMIT = 0.4
filtered_results = filter(lambda x: x[2].fitness > FITNESS_LIMIT, results)

In [ ]:
def generate_ref_point_cloud(transform):
    ref_pcd_temp = copy.deepcopy(ref_pcd)
    return ref_pcd_temp.transform(transform)

In [ ]:
pcd_all = [generate_ref_point_cloud(reg_res.transformation) for _,_,reg_res in filtered_results]
o3d.visualization.draw_geometries([pcd, *pcd_all])

In [ ]:
# find the best by fitness
all_fitness = [res[2].fitness for res in results]
best_pcd_idx = all_fitness.index(max(all_fitness))

In [ ]:
pcd_best = generate_ref_point_cloud(results[best_pcd_idx][2].transformation)
o3d.visualization.draw_geometries([pcd, pcd_best])